In [40]:
# Import libraries
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.autograd.functional import jacobian

In [41]:
# define objective fucntion
f =lambda x: x[0] ** 2 + x[1] ** 2 + x[2] ** 2 

# define constraints
h1 = lambda x: ((x[0] ** 2) / 4) + ((x[1] ** 2) / 5) + ((x[2] ** 2) / 25) - 1
h2 = lambda x: x[0] + x[1] - x[2]

# initialize Variables
x = Variable(torch.tensor([0.1,1.,1.]), requires_grad=True)
eps= 1e-03

We have $n=3$ and $m=2$ here, so the DOF of systme is $n-m = 1$ \
meanwe have only one decision variable variable and two state variable \
let say: \
$x_1 $ here is the decision variable \
$ x_2$  and  $x_3$ are the state variables \


In [42]:
# Calculating Reduce Gradient using jacobian
def RJ(f,h1,h2,x):
  #compute Jacobian
  Jocobian = torch.zeros((3, 3))
  Jocobian[0] = jacobian(f, (x))
  Jocobian[1] = jacobian(h1, (x))
  Jocobian[2] = jacobian(h2, (x))

  # reduce Gradient variables
  dfdd = Jocobian[0,0]
  dfds = Jocobian[0,1:]
  dhds = Jocobian[1:,1:]
  dhdd = Jocobian[1:,0]

  #compute reduce Gradient 
  DfDd =  dfdd - torch.matmul(torch.matmul(dfds,torch.pinverse(dhds)),dhdd)
  return  DfDd, dfdd, dfds, dhds, dhdd

In [43]:
def LM(x):
  ''' Levenberg -Marquardt'''
  Lambda = 1. 
  norm = torch.norm(torch.tensor([h1(x),h2(x)]))
  while norm >1e-06 :
      ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)
      with torch.no_grad():
          # Newton Method
          x[1:] = x[1:] - torch.matmul(torch.matmul(torch.pinverse(torch.matmul(dhds.T,dhds) + Lambda* torch.eye(2)), dhds.T),torch.tensor([h1(x),h2(x)]))
      norm= torch.norm(torch.tensor([h1(x),h2(x)]))
  return x
  
def newX(x, alpha):
    updatedx = torch.zeros(3)
    ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)
    updatedx[0] = x[0] - alpha * ReduceJacobian
    updatedx[1:] = x[1:] + (alpha * (torch.matmul(torch.pinverse(dhds),dhdd))*ReduceJacobian)
    return updatedx


def lineSearch(x, t0=0.5, K=25):
    ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)
    alpha = 1
    i = 0
    func = f(newX(x, alpha))
    phi = f(x) - (t0 * alpha * (ReduceJacobian ** 2))
    while func > phi and i < K:
        alpha = 0.5 * alpha
        func = f(newX(x, alpha))
        phi = f(x)- (t0 * alpha * (ReduceJacobian ** 2))
        i += 1
    return alpha

In [48]:
def GRG(x,eps= 1e-03):
    ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)
    e = torch.norm(ReduceJacobian)
    x_val = [x.detach().numpy()]
    x= LM(x)
    x_val.append(x.detach().numpy())
    DfDd_val = [f(x).item()]
    alpha_val = [1]
    error_Val = [e]
    k = 0
    while e > eps:

        alpha = lineSearch(x) # step 4.1
        ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)

        with torch.no_grad():
            x[0] = x[0] - alpha * ReduceJacobian # setp 4.2
            x[1:] = x[1:] + (alpha * np.matmul(torch.pinverse(dhds) ,  dhdd) *  ReduceJacobian) # step 4.3

        x = LM(x)  # step 4.4
        e = torch.norm(ReduceJacobian)  #step 4.5

        # record values
        x_val.append(x.detach().numpy())
        DfDd_val.append(f(x).item())
        alpha_val.append(alpha)
        error_Val.append(e)
        k +=1        
    return x_val, DfDd_val, alpha_val, error_Val

In [49]:
GRG(x)

([array([-1.5737593 ,  1.3770875 , -0.19667158], dtype=float32),
  array([-1.5737593 ,  1.3770875 , -0.19667158], dtype=float32)],
 [4.411767959594727],
 [1],
 [tensor(0.0002)])